In [ ]:
def pprint(arr):
    print("type:{}".format(type(arr)))
    print("shape: {}, dimension: {}, dtype:{}".format(arr.shape, arr.ndim, arr.dtype))
    print("Array's Data:\n", arr)

import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load input data
# data = pd.read_csv("Dataframe_l1_special.csv")
#data = pd.read_csv("2_8.csv")
data = pd.read_csv("1_1_10p.csv")
data = data.values
N    = 10000

np.random.shuffle(data)

# Make training data balanced & Shuffle 
data1 = data[0:N,:]
data2 = data[N:2*N,:]
data3 = data[2*N:3*N,:]
data4 = data[3*N:4*N,:]
data5 = data[4*N:5*N,:]
data6 = data[5*N:6*N,:]
data7 = data[6*N:7*N,:]
data8 = data[7*N:8*N,:]
data9 = data[8*N:9*N,:]
data10 = data[9*N:10*N,:]

np.random.shuffle(data1)
np.random.shuffle(data2)
np.random.shuffle(data3)
np.random.shuffle(data4)
np.random.shuffle(data5)
np.random.shuffle(data6)
np.random.shuffle(data7)
np.random.shuffle(data8)
np.random.shuffle(data9)
np.random.shuffle(data10)


N_train = round(0.7*N)
N_val   = round(0.1*N)


data_train = np.concatenate((data1[0:N_train,:],data2[0:N_train,:],data3[0:N_train,:],data4[0:N_train,:],data5[0:N_train,:],data6[0:N_train,:],data7[0:N_train,:],data8[0:N_train,:],data9[0:N_train,:],data10[0:N_train,:]), axis=0)
data_val   = np.concatenate((data1[N_train:(N_train+N_val),:],data2[N_train:(N_train+N_val),:],data3[N_train:(N_train+N_val),:],data4[N_train:(N_train+N_val),:],data5[N_train:(N_train+N_val),:],data6[N_train:(N_train+N_val),:],data7[N_train:(N_train+N_val),:],data8[N_train:(N_train+N_val),:],data9[N_train:(N_train+N_val),:],data10[N_train:(N_train+N_val),:]), axis=0)
data_test  = np.concatenate((data1[(N_train+N_val):N,:],data2[(N_train+N_val):N,:],data3[(N_train+N_val):N,:],data4[(N_train+N_val):N,:],data5[(N_train+N_val):N,:],data6[(N_train+N_val):N,:],data7[(N_train+N_val):N,:],data8[(N_train+N_val):N,:],data9[(N_train+N_val):N,:],data10[(N_train+N_val):N,:]), axis=0)

x_train = data_train[:,0:140]
y_train = data_train[:,142]
x_val   = data_val[:,0:140]
y_val   = data_val[:,142]
x_test  = data_test[:,0:140]
y_test  = data_test[:,142]

In [ ]:
from keras.metrics import sparse_top_k_categorical_accuracy
from keras import optimizers

def plot_history(histories):    
    plt.figure(figsize=(8,6))    
    for name, history in histories:
        val = plt.plot(history.epoch, history.history['val_sparse_categorical_accuracy'], '--', label=name+' (Validation)')
        plt.plot(history.epoch, history.history['sparse_categorical_accuracy'], color=val[0].get_color(), label=name+' (Train)') 
        
        val = plt.plot(history.epoch, history.history['val_top2_acc'], 'r--', label=name+' Top 2 (Validation)')
        plt.plot(history.epoch, history.history['top2_acc'], color=val[0].get_color(), label=name+' Top 2 (Train)')        
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.xlim([0, max(history.epoch)])
    plt.grid()
    
def top2_acc(y_true, y_pred):
    return sparse_top_k_categorical_accuracy(y_true, y_pred, k=2) 
    
# Train Phase
model = keras.Sequential([
    keras.layers.Dense(140, activation=tf.nn.selu, input_shape=(140,),
                       kernel_initializer='he_normal', bias_initializer='he_normal',
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(110, activation=tf.nn.selu,
                       kernel_initializer='he_normal', bias_initializer='he_normal',
                       kernel_regularizer=keras.regularizers.l2(0.01)),   
    keras.layers.Dense(80, activation=tf.nn.selu,
                       kernel_initializer='he_normal', bias_initializer='he_normal',
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(60, activation=tf.nn.selu,
                       kernel_initializer='he_normal', bias_initializer='he_normal',
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(43, activation=tf.nn.softmax)
])


# Optimizer
adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.0, nesterov=True)
model.compile(optimizer = adam,
             loss       = 'sparse_categorical_crossentropy',
             metrics    = ['sparse_categorical_accuracy',top2_acc])

model.summary()
model_history = model.fit(x_train, y_train,
                          epochs          = 500,
                          batch_size      = 128,
                          validation_data = (x_val,y_val),
                          verbose         = 0)

plot_history([('mMAPS', model_history)])

# Test Phase
test_loss, test_acc, test_top2acc = model.evaluate(x_test, y_test)
print('Test Accuracy:', test_acc)
print('Test Top-2 Accuracy:', test_top2acc)